<a href="https://colab.research.google.com/github/Shahahaha11/LION_DEN/blob/main/trial2_DATA_CLEANING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name map


---


trial -> trying new variable

---


data_cleaning -> a separate file which ends with saving a pickle that you use in model.ipynb


---


model -> Main model with little extra code

---


model_testing -> Less code, optimized for testing

---


In [31]:
import pandas as pd
import numpy as np
file_path= "/content/austinHousingData.csv"
data = pd.read_csv(file_path)

In [54]:
data.columns

Index(['zpid', 'city', 'streetAddress', 'zipcode', 'description', 'latitude',
       'longitude', 'propertyTaxRate', 'garageSpaces', 'hasAssociation',
       'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa', 'hasView',
       'homeType', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'latestPriceSource', 'numOfPhotos',
       'numOfAccessibilityFeatures', 'numOfAppliances', 'numOfParkingFeatures',
       'numOfPatioAndPorchFeatures', 'numOfSecurityFeatures',
       'numOfWaterfrontFeatures', 'numOfWindowFeatures',
       'numOfCommunityFeatures', 'lotSizeSqFt', 'livingAreaSqFt',
       'numOfPrimarySchools', 'numOfElementarySchools', 'numOfMiddleSchools',
       'numOfHighSchools', 'avgSchoolDistance', 'avgSchoolRating',
       'avgSchoolSize', 'MedianStudentsPerTeacher', 'numOfBathrooms',
       'numOfBedrooms', 'numOfStories', 'homeImage', 'lat_round', 'lon_round'],
      dtype='object')

In [33]:
################ NEW VARIABLE TEST ######### TO MERGE ON ZIP ##############################

In [57]:
house_inventory = pd.read_csv("/content/Affordable_Housing_Inventory_20250327.csv")
house_inventory.columns

Index(['Contract_ID', 'Project_ID', 'Project_Name', 'Ownership', 'Developer',
       'Address', 'ZIP', 'Parcel_ID', 'Council_District', 'Unit_Type',
       'Housing_Tenure', 'Development_Status', 'Total_Units',
       'Affordable_Units', 'Total_ADA_Units', 'PSH_Units', 'MFI_20', 'MFI_30',
       'MFI_40', 'MFI_50', 'MFI_60', 'MFI_65', 'MFI_70', 'MFI_80', 'MFI_100',
       'MFI_120', 'Affordability_Start_Date', 'Affordability_Expiration_Date',
       'Affordability_Period', 'FIL_Status', 'CAL_FIL', 'Date_FIL_Recorded',
       'HDA', 'Private_Activity_Bonds', 'Affordability_Unlocked', 'SMART',
       'Dev_Agreement', 'Density_Bonus', 'FIL', 'LIHTC', 'GO_Bonds_2006',
       'GO_Bonds_2013', 'GO_Bonds_2018', 'GO_Bonds_2022', 'CDBG', 'CIP_Funds',
       'HOME', 'HTF', 'Project_Connect', 'City_Subsidy', 'Longitude',
       'Latitude'],
      dtype='object')

In [58]:
########### CHECKING IF MERGING IS RIGHT #########
missing_count =(data['zipcode'].isin(house_inventory['ZIP']) == False).sum()
print(missing_count)


2032


if we have n unique values in one of the columns in the new data set


In [52]:
house_inventory['lon_round'].duplicated().sum()


np.int64(2343)

In [59]:
house_inventory.isna().sum()

,0
Contract_ID,0
Project_ID,0
Project_Name,0
Ownership,17
Developer,14
Address,0
ZIP,2
Parcel_ID,36
Council_District,1
Unit_Type,5


In [60]:
house_inventory['Housing_Tenure'].unique()

array(['Rental', 'Ownership', 'Non-Residential', 'Mixed', nan],
      dtype=object)

In [61]:
house_inventory[house_inventory['Housing_Tenure']== 'Rental'].shape[0]

672

In [ ]:
house_inventory= house_inventory.set_index('ZIP')
house_inventory

In [ ]:
data['population'] = data['zipcode'].map(zp['population'])
data['density'] = data['zipcode'].map(zp['density'])

In [ ]:
zp = pd.read_csv("/content/uszips.csv")
zp['zip']

,zip
0,601
1,602
2,603
3,606
4,610
...,...
33778,99923
33779,99925
33780,99926
33781,99927


In [ ]:
zp[zp['city']== 'Austin']['zip'].nunique()

51

In [ ]:
########### CHECKING IF MERGING IS RIGHT #########
missing_count = (data['zipcode'].isin(zp['zip']) == False).sum()
print(missing_count)

0


In [ ]:
import pandas as pd
zp = zp.set_index('zip')
zp.head()

,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
zip,,,,,,,,,,,,,,,,,
601,18.18027,-66.75266,Adjuntas,PR,Puerto Rico,True,NaN,16721.0,100.2,72001,Adjuntas,"{""72001"": 98.74, ""72141"": 1.26}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
602,18.36075,-67.17541,Aguada,PR,Puerto Rico,True,NaN,37510.0,477.6,72003,Aguada,"{""72003"": 100}",Aguada,72003,False,False,America/Puerto_Rico
603,18.45744,-67.12225,Aguadilla,PR,Puerto Rico,True,NaN,48317.0,543.1,72005,Aguadilla,"{""72005"": 99.76, ""72099"": 0.24}",Aguadilla|Moca,72005|72099,False,False,America/Puerto_Rico
606,18.16585,-66.93716,Maricao,PR,Puerto Rico,True,NaN,5435.0,47.3,72093,Maricao,"{""72093"": 82.27, ""72153"": 11.66, ""72121"": 6.06}",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
610,18.29110,-67.12243,Anasco,PR,Puerto Rico,True,NaN,25413.0,264.4,72011,Añasco,"{""72011"": 96.71, ""72099"": 2.82, ""72083"": 0.37,...",Añasco|Moca|Las Marías|Aguada,72011|72099|72083|72003,False,False,America/Puerto_Rico


In [ ]:
data['population'] = data['zipcode'].map(zp['population'])
data['density'] = data['zipcode'].map(zp['density'])

In [ ]:
data[data['population'].isna()]['population'].sum()

np.float64(0.0)

In [ ]:
#USELESS FEATURES ## BUT WE DROP "city" and "zipcode" after using them. ## no dropping yet
useless_features= ['zipcode','zpid', 'city', 'streetAddress', 'zipcode', 'description','homeImage','homeType', 'latestPriceSource']

In [63]:
data.isna().sum()

,0
zpid,0
city,0
streetAddress,0
zipcode,0
description,2
latitude,0
longitude,0
propertyTaxRate,0
garageSpaces,0
hasAssociation,0


In [ ]:
data['latest_saledate']

,latest_saledate
0,2019-09-02
1,2020-10-13
2,2019-07-31
3,2018-08-08
4,2018-10-31
...,...
15166,2020-12-10
15167,2020-12-16
15168,2021-01-27
15169,2019-08-07


In [ ]:
data['latest_saledate'] = data['latest_saledate'].astype(str).str.strip()
data['latest_saledate'] = pd.to_datetime(data['latest_saledate'], errors='coerce')
print(data['latest_saledate'].isna().sum())

0


In [ ]:
data['latest_saledate'].head()

,latest_saledate
0,2019-09-02
1,2020-10-13
2,2019-07-31
3,2018-08-08
4,2018-10-31


In [ ]:
data = data.sort_values('latest_saledate').reset_index(drop=True)
data.isna().sum()

,0
zpid,0
city,0
streetAddress,0
zipcode,0
description,2
latitude,0
longitude,0
propertyTaxRate,0
garageSpaces,0
hasAssociation,0


In [ ]:
 # We have 958 days of analysis
 data['latest_saledate'].value_counts().sort_index().isna().sum()

np.int64(0)

In [ ]:
#################### 1 CITY ONLY ###############################
df= data[data['city'] == 'austin'].sort_values(by='latest_saledate')
df.select_dtypes(include='integer').columns

Index(['zpid', 'zipcode', 'garageSpaces', 'parkingSpaces', 'yearBuilt',
       'numPriceChanges', 'latest_salemonth', 'latest_saleyear', 'numOfPhotos',
       'numOfAccessibilityFeatures', 'numOfAppliances', 'numOfParkingFeatures',
       'numOfPatioAndPorchFeatures', 'numOfSecurityFeatures',
       'numOfWaterfrontFeatures', 'numOfWindowFeatures',
       'numOfCommunityFeatures', 'numOfPrimarySchools',
       'numOfElementarySchools', 'numOfMiddleSchools', 'numOfHighSchools',
       'avgSchoolSize', 'MedianStudentsPerTeacher', 'numOfBedrooms',
       'numOfStories'],
      dtype='object')

In [ ]:
data['latest_saledate'].dropna()

,latest_saledate
0,2018-01-22
1,2018-01-22
2,2018-01-22
3,2018-01-22
4,2018-01-23
...,...
15166,2021-01-27
15167,2021-01-28
15168,2021-01-29
15169,2021-01-29


In [ ]:
df.select_dtypes(include='object').columns

Index(['city', 'streetAddress', 'description', 'homeType', 'latestPriceSource',
       'homeImage'],
      dtype='object')

In [ ]:
################# FINAL DATAFRAME IN USE ################
df_1 = df.drop(columns=useless_features, errors='ignore')
df_1.shape

(15020, 41)

In [ ]:
# FINAL CHECK BEFORE MERGING
df_1.isna().sum()

,0
latitude,0
longitude,0
propertyTaxRate,0
garageSpaces,0
hasAssociation,0
hasCooling,0
hasGarage,0
hasHeating,0
hasSpa,0
hasView,0


In [ ]:
# Get Mortgage rate
import pandas as pd
MTG = pd.read_csv('/content/MORTGAGE30US.csv')

print(MTG.shape)

MPPSQ = pd.read_csv('/content/MEDLISPRIPERSQUFEE12420.csv')
print(MPPSQ.dtypes)

(160, 2)
observation_date           object
MEDLISPRIPERSQUFEE12420     int64
dtype: object


In [ ]:
############################################################################### Tranforming Mortgage Data #############################################################################################

In [ ]:
################## TREATING DATES ON EXTERNAL MACRO VARIABLES ##################
def parse_obs_date(date_str):
    d = str(date_str).strip().split()[0]
    if "/" in d:
        return pd.to_datetime(d, format='%d/%m/%Y', errors='raise')
    else:
        return pd.to_datetime(d, errors='raise')

MTG['observation_date'] = MTG['observation_date'].apply(parse_obs_date)
MPPSQ['observation_date'] = MPPSQ['observation_date'].apply(parse_obs_date)
MTG['MORTGAGE30US'].dtypes

dtype('float64')

In [ ]:
print(MTG.isna().sum())
print(MPPSQ.isna().sum())

observation_date    0
MORTGAGE30US        0
dtype: int64
observation_date           0
MEDLISPRIPERSQUFEE12420    0
dtype: int64


In [ ]:
df_2 = df_1.copy()

In [ ]:
##################################.MERGE. ####################################

import pandas as pd

df_2['latest_saledate'] = pd.to_datetime(df_2['latest_saledate'], errors='coerce')
df_2 = df_2.sort_values('latest_saledate')

# Read and prepare the MTG rates
MTG['observation_date'] = pd.to_datetime(MTG['observation_date'], format='%d/%m/%Y', errors='coerce')
MTG = MTG.sort_values('observation_date')

# Merge using asof so that each sale date gets the most recent MTG rate available
df_2 = pd.merge_asof(df_2, MTG, left_on='latest_saledate', right_on='observation_date', direction='backward')

# Optional: Remove the duplicate 'date' column if no longer need it
df_2.drop(columns=['observation_date'], inplace=True)
df_2.columns

Index(['latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'numOfPhotos', 'numOfAccessibilityFeatures',
       'numOfAppliances', 'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt',
       'livingAreaSqFt', 'numOfPrimarySchools', 'numOfElementarySchools',
       'numOfMiddleSchools', 'numOfHighSchools', 'avgSchoolDistance',
       'avgSchoolRating', 'avgSchoolSize', 'MedianStudentsPerTeacher',
       'numOfBathrooms', 'numOfBedrooms', 'numOfStories', 'population',
       'density', 'MORTGAGE30US'],
      dtype='object')

In [ ]:
# read variable, sort variable on date
MPPSQ['observation_date'] = pd.to_datetime(MPPSQ['observation_date'], format='%d/%m/%Y', errors='coerce')
MPPSQ = MPPSQ.sort_values('observation_date')

# Merge using asof so that each sale date gets the most recent MPPSQ available
df_2 = pd.merge_asof(df_2, MPPSQ, left_on='latest_saledate', right_on='observation_date', direction='backward')

# Optional: Remove the duplicate 'date' column if you no longer need it
df_2.drop(columns=['observation_date'], inplace=True)
df_2.columns

Index(['latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'numOfPhotos', 'numOfAccessibilityFeatures',
       'numOfAppliances', 'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt',
       'livingAreaSqFt', 'numOfPrimarySchools', 'numOfElementarySchools',
       'numOfMiddleSchools', 'numOfHighSchools', 'avgSchoolDistance',
       'avgSchoolRating', 'avgSchoolSize', 'MedianStudentsPerTeacher',
       'numOfBathrooms', 'numOfBedrooms', 'numOfStories', 'population',
       'density', 'MORTGAGE30US', 'MEDLISPRIPERSQUFEE12420'],
      dtype='object')

In [ ]:
df_2[['latest_saledate','MORTGAGE30US','MEDLISPRIPERSQUFEE12420']]

,latest_saledate,MORTGAGE30US,MEDLISPRIPERSQUFEE12420
0,2018-01-22,4.04,163
1,2018-01-22,4.04,163
2,2018-01-22,4.04,163
3,2018-01-22,4.04,163
4,2018-01-23,4.04,163
...,...,...,...
15015,2021-01-27,2.77,226
15016,2021-01-28,2.73,226
15017,2021-01-29,2.73,226
15018,2021-01-29,2.73,226


In [ ]:
df_2.isna().sum()

,0
latitude,0
longitude,0
propertyTaxRate,0
garageSpaces,0
hasAssociation,0
hasCooling,0
hasGarage,0
hasHeating,0
hasSpa,0
hasView,0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# change path as desired, use the same path in the model file
df_2.to_pickle("/content/drive/MyDrive/lions_den_data/df_2_trial.pkl")

In [ ]:
## run -> on next script -> df_2 = pd.read_pickle('/content/drive/MyDrive/lions_den_data/df_2.pkl')
